# 0. Введение

В предыдущих тетрадках мы прошли все этапы пайплайна по обработке документов - от подготовки данных для обучения моделей до написания метрик. Нам остается только собрать все кусочки в один единый сервис. 

В этой тетрадке будем немного оптимизировать инференс, объединять детекцию, распознавание и извлечение сущностей и считать end-to-end метрики. План примерно такой: 

1. Инференс детектора текста;
2. Инференс распознавания текста; 
3. Инференс модели для линий; 
4. Объединение линий в параграфы; 
5. Объединение предыдущих четырех шагов в один метод по обработке изображения; 
6. Получить предсказание с помощью NER модели для распознанного текста
7. Подготовить данные и расчитать метрики
8. Собрать сервис на flask

Также на каждом этапе будем визуализировать результаты, чтобы убедиться, что мы ничего нигде не забыли. Поехали! 

Следующие несколько ячеек будут общими для всех ноутбуков.

* Эта ячейка настраивает отображение ipython widgets

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%matplotlib inline

* Подключаем Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

* Указываем путь к папке с кодом: 

In [ ]:
repo_folder = '/content/drive/MyDrive/DeepLearning2/'

* Устанавливаем зависимости

In [ ]:
reqs_path = repo_folder + 'IntelligentDocumentProcessing/requirements.txt '
!pip3 install -r {reqs_path}

* Подключаем WandB

In [ ]:
import wandb
wandb_key = open('/content/drive/MyDrive/ssh/wandbkey.txt').read().strip()
wandb.login(key=wandb_key)

* Подключаем утилиты для этого ноутбука

In [ ]:
import sys
base_folder = repo_folder + 'IntelligentDocumentProcessing/Resources/e_Service_Deployment/'  # import utils
sys.path.append(base_folder)
sys.path.append(repo_folder + 'IntelligentDocumentProcessing/Resources/')  # from a_Text_Detection.utils import
sys.path.append(repo_folder)  # from IntelligentDocumentProcessing.Resources.a_Text_Detection.utils import

# 1: Перевод изображения в текст

## 1.1: Вход в пайплайн: изображение

In [ ]:
import cv2
import matplotlib.pyplot as plt


image_fpath = base_folder+'ner_sample/821284f7-4c42-491e-b85d-9d37a2ce7a56.jpeg'

image = cv2.imread(image_fpath)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(15, 12))
plt.imshow(image)
plt.show()

In [ ]:
from c_Layout_Analisys.utils import resize_aspect_ratio

device = 'cpu'
max_image_size = 2048

image_resized, _, _ = resize_aspect_ratio(image, square_size=max_image_size, interpolation=cv2.INTER_LINEAR)

## 1.2: Детекция текста

Сначала на изображении найдем все локации, где есть текст и границы текста.

Для этого возьмем модель, которую мы обучили для этой цели, и применим ко входному изображению.

In [ ]:
import torch


model_fpath = '/content/drive/MyDrive/DeepLearning2/td.jit'
text_detection_model = torch.jit.load(model_fpath, map_location=torch.device(device))
text_detection_model.eval();

### Задача 1 (разминочная)

Метод для инференса детекции текста мы уже писали в первой тетрадке, поэтому необходимо просто вставить код из первой тетрадки сюда (только название должно быть `text_detection_inference`). 

#### Код

In [ ]:
from typing import Union, List

import albumentations as A
from albumentations import BasicTransform, Compose, OneOf
from albumentations.pytorch import ToTensorV2
import numpy as np
import torch.nn as nn

from a_Text_Detection.utils import Postprocessor, DrawMore

# КОД ДЛЯ СТУДЕНТА
# сюда необходимо вставить код для инференса модели из тетрадки по детекции
def text_detection_inference(
    model: nn.Module, 
    image: np.ndarray, 
    transform: Union[BasicTransform, Compose, OneOf],
    postprocessor: Postprocessor,
    device: str = 'cpu',
) -> List[np.ndarray]:
    pass

transform = ...
postprocessor = ...

#### Проверка

In [ ]:
pred_bboxes = text_detection_inference(text_detection_model, image_resized, 
                                       transform, postprocessor, device)

Тут и далее мы будем сохранять результаты разных этапов обработки документа в папку `results`, поэтому необходимо для начала ее создать. 

In [ ]:
import os

os.mkdir('results/')

In [ ]:
countours_result = DrawMore.draw_contours(image_resized, pred_bboxes, thickness=2, color=(0, 0, 255))
out_image_fpath = 'results/contours.png'
cv2.imwrite(out_image_fpath, countours_result)

plt.figure(figsize=(15, 12))
plt.imshow(countours_result.astype('int'))
plt.show()

## 1.3: Распознавание символов текста

На данный момент мы имеем изображене и прямоугольники (bounding box'ы), которые предсказала модель. Но на вход в OCR мы подаем вырезанные небольшие изображения, поэтому их необходимо достать из исходного изображения: 

In [ ]:
from utils import prepare_crops

crops = prepare_crops(image_resized, pred_bboxes)

### Задача 2 (тоже разминочная)

Вырезанные изображения подготовлены, но помимо предобработки есть еще постобработка. Как мы помним, ее в случае OCR выполняет токенизатор, который будет переводить предсказания модели в символы. В следующей ячейке мы предлагаем вставить его реализацию из тетрадки по OCR:  

#### Код

In [ ]:
# КОД ДЛЯ СТУДЕНТА
# сюда необходимо вставить код токенайзера из тетрадки по распознаванию текста
class TokenizerForCTC:
    pass

#### Проверка

In [ ]:
punct = " !\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~«»№"
digit = "0123456789"
cr = "ЁАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяё"
latin = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
alphabet = punct + digit + cr + latin

tokenizer = TokenizerForCTC(list(alphabet))

На всякий случае продублируем проверку токенизатора, чтобы убедиться, что мы скопировали то, что надо: 

In [ ]:
correct_tensor = torch.tensor([132, 153, 149, 143, 152, 147, 164, 143, 156, 2]).unsqueeze(1)
encoded = tokenizer.encode('Tokenizer!')
assert len(encoded) == 2, 'Метод encode должен возвращать 2 элемента: тензор и длину последовательности.'
assert torch.equal(encoded[0], correct_tensor), 'Строка "Tokenizer!" закодирована неправильно.'
assert encoded[1] == 10, "Метод encode вернул неправильную длину последовательности."

decoded = tokenizer.decode([146, 146, 0, 143, 0, 150, 150, 0, 153])
assert decoded == 'helo', "Метод decode неправильно декодировал последовательность."

decoded = tokenizer.decode([146, 146, 0, 143, 0, 150, 150, 0, 150, 153])
assert decoded == 'hello', "Метод decode неправильно декодировал последовательность с повторяющимися символами. "

Далее нам понадобится модель OCR, которую вы обучили: 

In [ ]:
ocr_model_fpath = '/content/drive/MyDrive/DeepLearning/ocr.jit'
ocr_model = torch.jit.load(ocr_model_fpath, map_location=torch.device(device))
ocr_model.eval();

### Задача 3: Батчевание инференса распознавания текста

У вас уже есть код для инференса модели для одного изображения, но делать инференс по одному изображению вычислительно невыгодно, поэтому теперь для оптимизации необходимо реализовать инференс с изменяемым размером батча. Итак, алгоритм:
1. Разбить входящие изображения с помощью метода `batchings` (это функция-генератор, которая принимает на вход список объектов и размер батча, а возвращает с помощью `yield` батчи по очереди);
3. Каждую картинку в батче преобразовать с помощью `resize_by_height`;
4. Вычислить максимальную ширину изображения в батче; 
5. Добить все изображения в батче до одной ширины значениями `pad_value`. Можно использовать метод `torch.nn.functional.pad` или `cv2.copyMakeBorder`;
6. Привести все изображения к тензорам и объединить в один тензор через `torch.stack`;
7. Далее идет почти обычный инференс, только возвращать метод будет не строку, а список строк. 

#### Код

In [ ]:
import torch.nn.functional as F

from utils import batchings
from b_Optical_Character_Recognition.utils import resize_by_height

# КОД ДЛЯ СТУДЕНТА
def ocr_inference(
    model: nn.Module, 
    image: np.ndarray, 
    transform: Union[BasicTransform, Compose, OneOf],
    tokenizer: TokenizerForCTC, 
    device: str = 'cpu',
    batch_size: int = 1,
    target_height: int = 32,
    pad_value: int = 0
) -> List[str]:
    pass

transform = ...
labels = ...

In [ ]:
labels = ocr_inference(ocr_model, crops, transform, tokenizer, device, batch_size=8)

#### Проверка

In [ ]:
from b_Optical_Character_Recognition.utils import draw_predictions

out_image_fpath = 'results/ocr.png'
_ = draw_predictions(
    crops, 
    predicted_texts=labels, 
    path_to_save_image=out_image_fpath,
    max_elements_to_draw=16
)

image = cv2.imread(out_image_fpath)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(15, 15), dpi=150)
plt.imshow(image)
plt.show()

In [ ]:
from c_Layout_Analisys.utils import Word

words = [Word(bbox, label) for bbox, label in zip(pred_bboxes, labels) if len(label) > 0]

## 1.4: Сборка текста в строки и параграфы

Загружаем модель для предсказания строк, которая нам уже знакома. Напоминаем, что это модель с архитектурой DB (очень похожая на модель детекции текста), которая выдает нам линии, а линии необходимо собрать в параграфы. 

In [ ]:
line_model_path = '/content/drive/MyDrive/DeepLearning/la.jit'
line_model = torch.jit.load(line_model_path, map_location=torch.device(device))
line_model.eval();

### Задача 4:

Возьмем код из тетрадки по layout analysis с инференсом модели по предсказанию линий: 

#### Код

In [ ]:
from c_Layout_Analisys.utils import Line

# необходимо вставить сюда инференс из тетрадки по layout
# КОД ДЛЯ СТУДЕНТА
def line_detector_inference(
    model: nn.Module, 
    image: np.ndarray, 
    transform: Union[BasicTransform, Compose, OneOf],
    postprocessor: Postprocessor,
    device: str = 'cpu',
) -> List[Line]:
    pass

transform = ...
postprocessor = ...

#### Проверка

Проверим детектирование строк текста: 

In [ ]:
lines = line_detector_inference(line_model, image_resized, transform, postprocessor, device)

In [ ]:
from utils import group_words_by_lines_or_lines_by_paragraphs 
from c_Layout_Analisys.utils import sort_boxes

# сгруппируем слова в линии по IOU
h, w, _ = image_resized.shape
lines = group_words_by_lines_or_lines_by_paragraphs(words, lines, w, h)
lines = [line for line in lines if len(line.items) > 0]
for line in lines:
    line.items = sort_boxes(line.items, sorting_type = 'left2right')  # сортировка слева направо
    line.label = ' '.join([word.label.strip() for word in line.items])

In [ ]:
lines_result = DrawMore.draw_contours(image_resized, [line.bbox for line in lines], thickness=2)
out_image_fpath = 'results/lines.png'
cv2.imwrite(out_image_fpath, lines_result)

plt.figure(figsize=(15, 12))
plt.imshow(lines_result.astype('int'))
plt.show()

Загрузим объект класса `ParagraphFinder` и проверим сборку параграфов текста:

In [ ]:
import math

import dill
from sklearn.cluster import DBSCAN

from c_Layout_Analisys.utils import sort_boxes_top2down_wrt_left2right_order, sort_boxes, fit_bbox, Paragraph


with open('/content/drive/MyDrive/DeepLearning/paragraph_finder.pkl', 'rb') as r:
    paragraph_finder = dill.load(r)

In [ ]:
paragraphs = paragraph_finder.find_paragraphs(lines)

In [ ]:
for para in paragraphs:
    para.label = ' '.join([line.label.strip() for line in para.items])

In [ ]:
para_result = DrawMore.draw_contours(image_resized, [para.bbox for para in paragraphs], thickness=2)
out_image_fpath = 'results/paragraphs.png'
cv2.imwrite(out_image_fpath, para_result)

plt.figure(figsize=(15, 12))
plt.imshow(para_result.astype('int'))
plt.show()

## 1.5: Сборка end-to-end OCR

Мы проверили, что каждый отдельный модуль приложения работает. Теперь сделаем функцию, которая на вход будет принимать документ, а на выходе будет выдавать текст.

In [ ]:
from utils import visualize_e2e

out_image_fpath = 'results/end2end.png'
font_path = repo_folder + 'IntelligentDocumentProcessing/Resources/b_Optical_Character_Recognition/resources/fonts/times.ttf'
_ = visualize_e2e(image_resized, paragraphs, font_path=font_path,
                  fontsize=20, font_color=(0, 0, 0), thickness=2, show_words=True, 
                  show_lines=True, show_groups=True, path_to_save_image=out_image_fpath)

image = cv2.imread(out_image_fpath)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(15, 12), dpi=150)
plt.imshow(image)
plt.show()

### Задача 4. Полный пайплайн распознавания документа

У вас есть все готовые методы для распознавания - осталось только собрать их в один метод. Параметры метода, кроме `image` и `device`, вы определяете сами, а на выходе должен получиться список DTO типа `Paragraph`. Что должно быть внутри метода: 

1. Инференс модели детектора текста - метод `text_detection_inference` в помощь;
1. Вызов метода `prepare_crops`, который вырезает из изображения прямоугольники с текстом; 
1. Инференс модели распознавания текста - метод `ocr_inference`;
1. Инференс модели, которая находит линии - метод `line_detector_inference`;
1. Объединение слов и линий с помощью метода `group_words_by_lines_or_lines_by_paragraphs`;
1. Сортировка слов в линиях;
1. Объединение линий в параграфы с помощью `paragraph_finder`.  


#### Код

In [ ]:
# КОД ДЛЯ СТУДЕНТА
def recognition_pipeline(
    image: np.ndarray,
    device: str,
    **kwargs
) -> List[Paragraph]:
    pass

#### Проверка

In [ ]:
full_pipeline_paragraphs = recognition_pipeline(
    image=image_resized, 
    device=device,
    detection_model=text_detection_model,
    detection_transform=transform,
    detection_postprocessor=postprocessor,
    line_model=line_model,
    line_transform=transform,
    line_postprocessor=postprocessor,
    paragraph_model=paragraph_finder,
    ocr_model=ocr_model,
    ocr_transform=transform,
    ocr_tokenizer=tokenizer,
    ocr_batch_size=8
)

for para in full_pipeline_paragraphs:
    for i, line in enumerate(para.items):
        print(i, line.label)

В следующей ячейке будут сравниваться параграфы, которые мы получили из метода `recognition_pipeline`, и те, которые мы получили, прогоняя модели по отдельности (в переменной `paragraphs`). 

In [ ]:
para_msg = 'Количество параграфов после Е2Е не совпадает с количеством параграфов в paragraphs.'
assert len(paragraphs) == len(full_pipeline_paragraphs), para_msg
for para, fp_para in zip(paragraphs, full_pipeline_paragraphs):
    label_msg = 'Текст каждого параграфа должен совпадать.'
    assert para.label == fp_para.label, label_msg
    bbox_msg = 'Bounding box каждого параграфа должен совпадать.'
    assert np.array_equal(para.bbox, fp_para.bbox), bbox_msg
    len_msg = 'Количество линий в каждом параграфе должно совпадать.'
    assert len(para.items) == len(fp_para.items), len_msg
    for line, fp_line in zip(para.items, fp_para.items):
        label_msg = 'Текст каждой линии должен совпадать.'
        assert line.label == fp_line.label, label_msg
        bbox_msg = 'Bounding box каждой линии должен совпадать.'
        assert np.array_equal(line.bbox, fp_line.bbox), bbox_msg
        for word, fp_word in zip(line.items, fp_line.items):
            label_msg = 'Текст каждого слова должен совпадать.'
            assert word.label == fp_word.label, label_msg
            bbox_msg = 'Bounding box каждого слова должен совпадать.'
            assert np.array_equal(word.bbox, fp_word.bbox), bbox_msg

### Задача 5. OCR на строках

Вы обучали OCR не просто на отдельных словах или парах слов, а на целых строках до 10-11 слов. Давайте теперь посмотрим, как эта модель себя поведет, если использовать ее на целых строках.

Вам необходимо написать метод, который будет аналогичным методу `recognition_pipeline`, но только OCR будет применяться к задетектированным строкам. 

Также параметры данного метода вы выбираете самостоятельно (кроме `image` и `device`). 

#### Код

In [ ]:
# КОД ДЛЯ СТУДЕНТА
def line_recognition_pipeline(
    image: np.ndarray,
    device: str,
    **kwargs
) -> List[Paragraph]:
    pass

#### Проверка

In [ ]:
line_pipeline_paragraphs = line_recognition_pipeline(
    image=image_resized,
    line_model=line_model,
    line_transform=transform,
    line_postprocessor=postprocessor,
    paragraph_model=paragraph_finder,
    ocr_model=ocr_model,
    ocr_transform=transform,
    ocr_tokenizer=tokenizer,
    ocr_batch_size=8,
    device=device
)

for para in line_pipeline_paragraphs:
    for i, line in enumerate(para.items):
        print(i, line.label)

## 1.6. Применение модели NER для полученного текста

Теперь наша задача запустить полный пайплайн, где на входе мы отправляем картинку, а на выходе имеет сущности с их координатами.

Поэтому выделим два основных компнонета: 

* `OCR Pipeline`
* `NER Model`


C `OCR` нам теперь все понятно, соберем только все вместе и получим текст с которым будем работать в `NER` на примере одной картинки. 


In [ ]:
device = 'cuda:0'
max_image_size = 2048

image_fpath = './team_idp/ocr_service/ner_sample/821284f7-4c42-491e-b85d-9d37a2ce7a56.jpeg'

image = cv2.imread(image_fpath)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

image_resized, _, _ = resize_aspect_ratio(image, square_size=max_image_size, interpolation=cv2.INTER_LINEAR)

In [ ]:
line_pipeline_paragraphs = line_recognition_pipeline(
    image=image_resized,
    line_model=line_model,
    line_transform=transform,
    line_postprocessor=postprocessor,
    paragraph_model=paragraph_finder,
    ocr_model=ocr_model,
    ocr_transform=transform,
    ocr_tokenizer=tokenizer,
    ocr_batch_size=8,
    device=device
)

rec_text = " ".join([line.label for para in line_pipeline_paragraphs for i, line in enumerate(para.items)])

#### Визуализируем исходный документ и заодно посмотрим на текст из OCR Pipeline


In [ ]:
image = cv2.imread(image_fpath)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(15, 12))
plt.imshow(image)
plt.show()
print()
print("Recognized text: ")
print()
print(rec_text)

Как и говорилось раньнее к этому занятию у вас должна быть обучена `NER` модель на основе датасета `RuRed`. Загружаем лучший ваш чекпоинт, сконвертированный в `jit`. 


In [ ]:
rec_text = """
  Владимиру Комлеву вручили первую карту игрока
  В НХЛ на базе «Мира» .
  22 декабря, в день 70летия российского хоккея, был праздник.
  В генеральный директор АО «КПК» Владимир Комлев стал чеспионом.
  В первым держателем карты игрока Ночной Хоккейнойв Лиги (НХЛ), выпущенной на базе платежной системы
  «Мир», Карту Владимиру Комлеву вручили президент Обанка «Югра» Алексей Нефедов и президент Ночной. 
  """

In [ ]:
model = torch.jit.load("./drive/MyDrive/weights/ner_rured.jit")

In [ ]:
from ner_model import inference_ner_model
from ipymarkup import show_span_line_markup

# Готовим примеры для подачи в датасет, оставляем формат, который был использован при обучении, но без сущностей
samples = [((0, len(rec_text), rec_text),[])]

result = inference_ner_model(samples, model, "cpu", batch_size = 4, num_workers = 4)

for sample_predictions in result:
    show_span_line_markup(*sample_predictions)
    print()

### Итого:
  - Мы собрали рабочий пайплайн, котрый умеет превращать картинку в текст и извлекать из текста некоторый набор сущностей.

  - Мы МОЛОДЦЫ!

  - Однако есть еще работа, которую мы должны выполнить. О ней речь пойдет в следующей главе этой тетрадки.

# 2. Тестирование итогового пайплайна структурирования информации



##  2.1. Подготовка данных для тестирования
И снова нам нужно готовить данные, а именно сопоставить:
- С точки зрения распознавания
  - Изображение
  - Предсказанный текстовый слой для изображения
- С точки зрения извлечения:
  - Исходный текст новости
  - Разметку для исходного текста 
  - Предсказания `NER` модели

Все это нам нужно для того, что замерить качество структурирования информации полным.

In [ ]:
from glob import glob

def map_images_and_annotation(annotation_path: str, images_path: str):
    
    annotation_path_mask = os.path.join(annotation_path, "*.ann")
    annot_files = glob(annotation_path_mask)
    mapping = []

    for ann_file in annot_files:
        _, name = os.path.split(ann_file)
        id_file = name.split("_")[0]
        text_file = f"{ann_file[:-4]}.txt"
        image_paths = glob(os.path.join(images_path, f"{name[:-4]}*.jpg"))          
        mapping.append((ann_file, text_file, image_paths))
        
    return mapping
            
            
mapping_markup_image = map_images_and_annotation(
    "team_idp/ner/RuRED-splitted/test", 
    "../one_column/test/"
)

Небольшое пояснение: маппинг возвращает список изобращений по той причине, что часть новостей весьма длинные и поэтому были размещены на нескольких страницах.

In [ ]:
mapping_markup_image[0]

Считаем разметку из пары файлов:  
- `document1.ann`
- `doсument1.txt`

Этот метод уже был имплементирован у нас в тетрадке по `NER`.  Я перенес его в утилиты, так что воспользуемся готовым.

In [ ]:
from ner_model import read_annotation_pair

read_annotation_pair(*mapping_markup_image[0][:2])

### Задача 6: Подокументная обработка полным пайплайном

Сейчас у нас есть одна нестыковка: пайплайн `OCR` наботает на уровне страницы, модель `NER` на уровне предложений, пора это все привести к общему знаменателю, а именно - **к документу**.

Для этого нам потребуется:
- прогонять все изображения одного через `OCR` (это у нас практически готово, не хватает цикла)
- объединять распознынный текст с разных изображений (это легко!)
- сегментировать текст на предложения для подачи в `NER` (это я сделал за вас)
- прогонять примеры через модель (тут мы уже постарались, так что просто импортируем)
- объединять сущности по предложениям в сущности по документам (здесь немного покодим)

Так что вперед!

Для начала посмотрим как работает сегментация и что она нам вернет.

In [ ]:
from ner_model import sentence_split

text = "СЕйчасс Будем проверять тесст после расп0завания на т0 как он делитьСя на предло)|(ения. В лучшем случае это будет так ."

expected_result = [
    (
        (0, 89, 'СЕйчасс Будем проверять тесст после расп0завания на т0 как он делитьСя на предло)|(ения.'), # предложения и его координаты в документе
        [] # список в котором должны находиться сущности, если бы тренировали модель, на инференсе это просто легаси, чтобы не переделывать CustomDataset
     ),
    (
        (89, 121, 'В лучшем случае это будет так .'),  # предложения и его координаты в документе
        [] # легаси
     )
]

current_result = sentence_split(text)

assert expected_result == current_result, "Не совпадает с ожидаемым результатом, нужно проверить выходной формат данных"

#### Подзадача 1: Объединение предсказаний `NER` модели

Сейчас сущности имеют координаты в символах относительно каждого предложения, а нужно иметь набор сущностей на целый документ. Предложения пусть объединяются через пробел.

##### Код

In [ ]:
# КОД ДЛЯ СТУДЕНТА

def merge_predictions(document_predictions: List[Tuple[str, list]]) -> Tuple[str, list]:
    markup = []
    text = ""
    # 
    # Дополнение по коду
    # 
    return text, markup

##### Проверка

In [ ]:
## Проверка имплементации
 
samples = [("Есть два предложения: со странными сущностями.", [(5, 9, "Числительное")]), ("Нужно проверить что объединение корректно", [(0, 6 , "Глагол")])]
expected_result = ("Есть два предложения: со странными сущностями. Нужно проверить что объединение корректно", [(5, 9, "Числительное"), (47, 53 , "Глагол")])

current_result = merge_predictions(samples)

assert current_result == expected_result, "Скорее всего не совпали координаты сущностей после смещения"

#### Подзадача 2: Прогон пайплайна 

Как и говорилось выше нам нужно собрать в одном месте 4 абстракции, для дальнейшей оценки качества сервиса (сами изображения нам уже не пригодятся):
- Исходный текст новости (есть)
- Предсказанный текстовый слой для изображения (запустим `OCR`)
- Разметку для исходного текста (есть)
- Предсказания модели извлечения (запустим `NER`)


##### Код

In [ ]:
# КОД ДЛЯ СТУДЕНТА
information_per_document = [
##  ( исходный текст, исходная раметка, текст после распознавания, предсказанные сущности )  
]

for ann_file, text_file, image_fpaths in mapping_markup_image:
    
    ## заводим цикл на список путей до изображения 
    
        ## читаем изображение в память

        ## меняем каналы

        ## ресайзим 

        ## вызов пайплайна распознавания для каждого изображения
        
        ## объединияем все строки
    
    ## получение полнотекста для документа
    
    ## сегментация и форматирование примеров для подачи в модель извлечения 
    
    ## инференс NER модели
    
    ## объединение выхода из модели извлечения
    
    ## чтение разметки и исходных текстов
    pass
    

##### Проверка

 Так как для инференса вы будете использовать модели, которые обучали самостоятельно, данное задание можно проверить только формально, по структуре данных. Поэтому возьмем один документ и проверим какие типы лежат на каждой из позиций тупла

In [ ]:
first_sample = information_per_document[0]
assert (len(first_sample) == 4) and \             # должно быть 4 абстракции
        isinstance(first_sample[0], str) and \    # Исходный текст новости 
        isinstance(first_sample[2], str) and \    # Предсказанный текстовый слой для изображения
        isinstance(first_sample[1], list) and \   # Разметк для исходного текста 
        isinstance(first_sample[3], list), \      # Предсказания модели извлечения (запустим `NER`)
        "Структура данных не соответствует ожидаемой"

In [ ]:
information_per_document[0]

## 2.2. Замер метрик

Будем реализовытать **жесткую** и единственно доступную метрику оценки качества работы сервиса: сверим сколько сущностей было в разметке и сколько сущностей предсказал наш набор моделей. Корректным ответом будет являться тот спан сущности, текст в котором частично/полностью совпадает с ground truth, кроме того он должен иметь аналогичный тип сущности.

### Задача 7: Оценка точности на тестовом наборе

#### Подзадача 0: Метод для очистки и исправления текстов сущностей (в общем виде)

Перед тем как сравнивать тексты сущностей, давайте попробуем их немного почистить, то есть подкорректровать. Ясно, что некорректный порядок токенов мы поправить не сможем, а вот регистр или частотную замену (путаницу) НУЛЯ и заглавной буквы O мы можем поправить. Имлементируйте метод text_precessing согласно ошибкам, которые вы видите в коде для проверки.

**HINT**: а еще можете воспользоваться знанием того, как ошибается ваш OCR


##### Код

In [ ]:
import re

# КОД ДЛЯ СТУДЕНТА
def text_precessing(text: str) -> str:
    # 
    # Дополнение по коду
    # 
    return text

##### Проверка

In [ ]:
## Проверка имплементации
recognized_text = "0дно дел0 простo ПРИВ0ДИТЬ все к НИжнему РЕГИСТРУ, с0всем другое - провести грамотный анализ ошибок модели ocr"

expected_result = "одно дело простo приводить все к нижнему регистру, совсем другое - провести грамотный анализ ошибок модели ocr"

processed_text = text_precessing(recognized_text)

assert expected_result == processed_text, "Кажется нужно еще подумать на постобработкой"

In [ ]:
def get_ents_texts(text: str, markup: list):
    """
    Скипаем из разметки и предсказаний координаты и получаем список кортежей (тип сущности, текст сущности)
    """
    return [(t, text_precessing(text[s:e])) for i, (s, e, t) in enumerate(markup)]

#### Подзадача 1: Метод оценки точности для одного документа

Для дальнейшей агрегации результатов будем возвращать количество "попаданий" и количество сущностей из исходников разметки.

Что имплементируем? Метод `accuracy_per_doc`, который позвовляет узнать насколько точно мы струтурировали информацию в рамках одного документа. Для этого реализуем:
- получение текстов сущностей из разметки
- получение текстов сущностей из предсказаний модели
- сравнение текстов сущностей - полный перебор
  - сущность считается корректно предсказанной если дистанция Левенштейна меньше или равна заданному параметру (при нахождении пары сущности удаляются из обоих списков)
  - в противном случае не считаем сущность корректно предсказанной и не учитываем ее


##### Код

In [ ]:
from Levenshtein import distance

# КОД ДЛЯ СТУДЕНТА
def accuracy_per_doc(
    origin_text: str, markup: list, rec_text: str, predictions: list, lev_dist: int = 1
):
    gold_ents = get_ents_texts(origin_text, markup) # тексты сущностей из разметки
    predict_ents = get_ents_texts(rec_text, predictions) # тексты предсказанных сущностей 
    same_ent = 0 # счетчик для корректных сущностей
    # 
    # Дополнение по коду
    # 
    return same_ent, len(gold_ents)

##### Проверка

In [ ]:
## Проверка имплементации

gold_text = 'Проверка двух сущностей'
gold_markup = [(0, 8, "Test type 0"), (9, 13, "Test type 1")]
rec_text = 'Проветка двух сущностей'
predict_markup = [(0, 8, "Test type 0"), (9, 13, "Test type 1")]

TP, n_ents = accuracy_per_doc(gold_text, gold_markup, rec_text, predict_markup, lev_dist = 0)
expected_result = (1, 2)
assert (TP, n_ents) == expected_result, "Accuracy is not correct"

TP, n_ents = accuracy_per_doc(gold_text, gold_markup, rec_text, predict_markup, lev_dist = 1)
expected_result = (2, 2)
assert (TP, n_ents) == expected_result, "Accuracy is not correct"

### 2.2.1. Оценка моделей на тестовом датасете

Осталось пропустить информацию из всех документов через нашу метрику и узнать сколько же правильных ответов мы дали.

In [ ]:
def dataset_accuracy(information_per_document: dict, lev_dist: int ):
    match, all_ents = 0, 0
    for sample_info in information_per_document:
        match_doc, n_gold_ents = accuracy_per_doc(*sample_info, lev_dist)
        match += match_doc
        all_ents += n_gold_ents
    print(f"Accuracy: {round(match / all_ents * 100, 2)} with Lev distance: {lev_dist}")
    return match, all_ents


In [ ]:
_ = dataset_accuracy(information_per_document, 0)
print()
_ = dataset_accuracy(information_per_document, 1)
print()
_ = dataset_accuracy(information_per_document, 2)

# 3. Flask App

**Flask** - это веб-фреймворк, написанный на языке **Python**, предназначенный для создания веб-приложений. Он обеспечивает гибкость и имеет низкий порог вхождения, кроме того на нем написана уже не одна тысяча веб-сервисов и поэтмоу в сети найдется ответ на любой ваш вопрос. **Flask** — это расширяемая система, которая не обязывает использовать конкретную структуру директорий и не требует сложного шаблонного кода перед началом использования.

### Задача 8: "Создание приложения на движке Flask" 

####  Подзадача 0: "Сборка класса Pipeline с основным методом predict" 

Краткая постановка задачи:
* На вход принимает считанный в память объект изображения для распознавания и извлечения
* На выходе набор сущностей по типам с их текстами


##### Код

In [ ]:
# КОД ДЛЯ СТУДЕНТА

class Pipeline:
    
    def __init__(self):
        """
        Здесь нужно проинициализировать все модели, с помощью которых мы будем
        извлекать информацию и распознавать документы.
        """
        pass

    def predict(self, image) -> dict:
        """
        Изображение уже в памяти. Ресайзим, детектируем, распознаем, собираем в единый текст.
        Сегментируем на предложения, извлекаем, объединяем и форматирем в словарь.
        Return:  {
            "text" : "Успешные результаты распознавания текста"
            "entities: [("Сущность 1", "Успешные результаты"), ("Сущность 2", "распознавания текста")]
        }
        Поле сущности так можете дополнять (координаты в текста/на исхображении),
        если захотите визуализировать результаты.
        """
        pass

##### Проверка

In [ ]:
## Проверка имплементации
pipe = Pipeline()
model_result = pipe.predict(image)

assert all([True if i in {"recognized_text", "entities"} else False for i in model_result.keys()]), "Some keys not found in model result"

###   Реализация методов сервиса

* **version()** - метод **GET**, возвращает версию сервиса
* **health()** - метод **GET**, возвращает статус сервиса (Работает / Не работает)
* **predict()** - метод **POST**, на вход принмиает запрос - **multipart** - состоящий из файла (изображение для распознавания и извлечения) и дополнительных тезнических параметров



In [ ]:
import json
import numpy as np
from time import time
from flask import Flask, jsonify, make_response, request
from service_utils import create_ok_response, create_error_response

COMMON_VERSION = "0.0.1"
DEFAULT_HOST = "0.0.0.0"
DEFAULT_PORT = 5000


def create_app():
    
    try:
        pipe = Pipeline()
        
    except Exception as e:
        raise Exception(f"Can not load Pipeline: {e}")

    app = Flask(__name__)

    @app.route("/version", methods=["GET"])
    def version():
        version_data = {
            "common": COMMON_VERSION
        }
        return make_response(jsonify({"version": version_data}), 200)

    @app.route("/health", methods=["GET"])
    def health():
        output_data = {
            "health_status": "running"
        }
        return make_response(jsonify(output_data), 200)

    @app.route("/predict", methods=["POST"])
    def predict():
        
        received_image = request.files.get("image")
        if not received_image:
            return make_response(jsonify({
                    "errorMsg": "No file with key \"image\" was found"
                }), 400)
        
        image_bytes = received_image.read()

        
        req_params = request.form.get("requestParameters")
        if not req_params:
            return make_response(jsonify({
                    "errorMsg": "Expected key \"requestParameters\", but not found"
                }), 400)
        
        input_params = json.loads(req_params)


        for param in ["msgId", "workId", "msgTm"]:
            if param not in input_params:
                return make_response(jsonify({
                    "errorMsg": f"Form key requestParameters/\"{param}\" is not set!"
                }), 400)

        try:
            
            t_start = time()
            
            image = np.fromstring(image_bytes, np.uint8)
            
            model_result = pipe.predict(image)

        except Exception as e:
            output_data = create_error_response(
                msg_id=input_params["msgId"],
                work_id=input_params["workId"],
                error_msg=str(e)
            )
            return make_response(jsonify(output_data), 500)

        t_end = time()

        output_data = create_ok_response(
            msg_id=input_params["msgId"],
            work_id=input_params["workId"],
            model_result=model_result,
            model_time=t_end - t_start
        )

        return make_response(json.dumps(output_data, ensure_ascii=False), 200)

    return app


if __name__ == "__main__":
    app = create_app()
    app.run(host=DEFAULT_HOST, port=DEFAULT_PORT, threaded=False)

Теперь нам нужно запустить наше веб приложение. Для удобства отладки также нужно прописать две базовых переменные среды

In [ ]:
!export FLASK_APP=flask_app.py  # так ак файл с приложение отличается от дефолтного названия app.py (но все в ваших руках)
!export FLASK_ENV=development  # чтобы все ошибки писались в лог приложения

Узнаем версию сервиса 

In [ ]:
!curl http://127.0.0.1:5000/version

Узнаем статус сервиса: поднят он или нет

In [ ]:
!curl http://127.0.0.1:5000/health

Отправим реальный запрос с картинкой и параметрам, получим фейковый результат работы сервиса

In [ ]:
!curl -F "image=@/home/jovyan/SorokinSA/DeepLearning/team_idp/ocr_service/ner_sample/821284f7-4c42-491e-b85d-9d37a2ce7a56.jpeg" -F  "requestParameters={\"msgId\": \"string\", \"msgTm\": \"2020-04-07T17:52:18.222Z\", \"workId\": \"s\"}" http://127.0.0.1:5000/predict
